# Bitki Hastalığı Tanıma Modelleri Karşılaştırmalı Analizi

In [1]:
# Gerekli kütüphanelerin yüklenmesi
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import torch
from torchvision import transforms, datasets, models
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("husl")

# 1. Veri Seti ve Modellerin Tanıtımı

In [2]:
# Veri yapılarımız
datasets_info = {
    "Internal Test": "Eğitimde kullanılan verinin test parçası",
    "External Test": "Tamamen bağımsız yeni veri seti"
}

models_info = {
    "DINOv2": "Facebook'un DINOv2 modeli (ViT tabanlı)",
    "EfficientB4": "EfficientNet-B4 mimarisi",
    "MobileNet": "MobileNetV2 (Hafif model)",
    "ResNet50": "Klasik ResNet50",
    "VGG16": "VGG16 (Derin CNN)"
}

# Veri yolları
DATA_PATHS = {
    "Internal Test": "/home/bilal/Desktop/Projeler/DuzenlenmisPG-tts/test",
    "External Test": "/home/bilal/Desktop/Projeler/external_test"
}

MODEL_PATHS = {
    "DINOv2": "/mnt/c/Users/Bilal/Downloads/PlantMaster_Model/PlantMaster_Model/modeller/dinov2/dinov2_best.pth",
    "EfficientB4": "/mnt/c/Users/Bilal/Downloads/PlantMaster_Model/PlantMaster_Model/modeller/efficientb4_output/efficientb4_final_model.pth",
    "MobileNet": "/mnt/c/Users/Bilal/Downloads/PlantMaster_Model/PlantMaster_Model/modeller/mobilnet/best_plant_disease_model_mobilenet.pth",
    "ResNet50": "/mnt/c/Users/Bilal/Downloads/PlantMaster_Model/PlantMaster_Model/modeller/resnet50/best_plant_disease_model_resnet.pth",
    "VGG16": "/mnt/c/Users/Bilal/Downloads/PlantMaster_Model/PlantMaster_Model/modeller/vgg/best_plant_disease_model_vgg.pth"
}

In [3]:
from torchvision.datasets import ImageFolder
# Transform işlemleri
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Veri setlerini yükleme
def load_datasets():
    loaded_datasets = {}
    for name, path in DATA_PATHS.items():
        try:
            dataset = ImageFolder(root=path, transform=test_transform)
            loaded_datasets[name] = dataset
            print(f"{name} verisi yüklendi. Örnek sayısı: {len(dataset)}")
        except Exception as e:
            print(f"{name} yüklenirken hata: {str(e)}")
    return loaded_datasets

datasets = load_datasets()

if "Internal Test" in datasets:
    class_names = datasets["Internal Test"].classes
    num_classes = len(class_names)
    print(f"\nSınıf İsimleri: {class_names}")
else:
    print("\n'Internal Test' verisi yüklenemedi.")

Internal Test verisi yüklendi. Örnek sayısı: 8802
External Test verisi yüklendi. Örnek sayısı: 207

Sınıf İsimleri: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Toma

In [6]:
from model_predictors import (
    DINOv2Predictor,
    EfficientB4Predictor,
    MobilenetPredictor,
    Resnet50Predictor,
    Vgg16Predictor
)

class ModelEvaluator:
    def __init__(self, model_name, model_path, class_names):
        self.model_name = model_name
        self.class_names = class_names
        self.predictor = self._load_predictor(model_path)
        self.transform = test_transform
    
    def _load_predictor(self, model_path):
        """Önceden yazılan predictor sınıflarını kullanarak model yükler"""
        if self.model_name == "DINOv2":
            return DINOv2Predictor(model_path, self.class_names)
        elif self.model_name == "EfficientB4":
            return EfficientB4Predictor(model_path, self.class_names)
        elif self.model_name == "MobileNet":
            return MobilenetPredictor(model_path, self.class_names)
        elif self.model_name == "ResNet50":
            return Resnet50Predictor(model_path, self.class_names)
        elif self.model_name == "VGG16":
            return Vgg16Predictor(model_path, self.class_names)
        else:
            raise ValueError(f"Bilinmeyen model: {self.model_name}")
    
    def evaluate(self, dataset):
        """Modeli veri seti üzerinde değerlendirir"""
        loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)
        y_true, y_pred = [], []
        
        with torch.no_grad():
            for images, labels in tqdm(loader, desc=f"{self.model_name} Değerlendirme"):
                # Predictor'ı kullanarak tahmin yapma
                batch_preds = []
                for img, label in zip(images, labels):
                    img_pil = transforms.ToPILImage()(img)
                    result = self.predictor.predict(img_pil)
                    batch_preds.append(result['class_index'])
                    y_true.append(label.item())
                
                y_pred.extend(batch_preds)
        
        accuracy = accuracy_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred)
        cr = classification_report(y_true, y_pred, target_names=self.class_names, output_dict=True)
        
        return {
            "accuracy": accuracy,
            "confusion_matrix": cm,
            "classification_report": cr,
            "true_labels": y_true,
            "pred_labels": y_pred
        }

In [ ]:
%%time
results = {}

for model_name, model_path in MODEL_PATHS.items():
    print(f"\n{'-'*50}")
    print(f"{model_name} modeli değerlendiriliyor...")
    
    evaluator = ModelEvaluator(model_name, model_path, class_names)
    model_results = {}
    
    for ds_name, dataset in datasets.items():
        print(f"\n{ds_name} verisi üzerinde test ediliyor...")
        model_results[ds_name] = evaluator.evaluate(dataset)
    
    results[model_name] = model_results
    print(f"{model_name} değerlendirmesi tamamlandı!")


--------------------------------------------------
DINOv2 modeli değerlendiriliyor...

Internal Test verisi üzerinde test ediliyor...


DINOv2 Değerlendirme:   2%|▏         | 5/276 [01:33<1:25:11, 18.86s/it]